CaféLocate

In Rostock, a lively city with a thriving academic community, CaféLocate is the go-to solution for coffee shop owners looking to strategically place their newly renovated café for optimal visibility and customer traffic

I import the necessary libraries such as pandas, folium, and KMeans

In [127]:
import pandas as pd
import folium
from sklearn.cluster import KMeans

Loading the CSV file

In [128]:
df = pd.read_csv('../Cafe_clastering/dataset/bibliotheken.csv')

In [129]:
df.head()

,latitude,longitude,uuid,kreis_name,kreis_schluessel,gemeindeverband_name,gemeindeverband_schluessel,gemeinde_name,gemeinde_schluessel,gemeindeteil_name,...,art,bezeichnung,traeger_bezeichnung,traeger_art,barrierefrei,oeffnungszeiten,telefon_festnetz,telefon_mobil,email,website
0,54.085840,12.135889,7a9d0038-4d54-11e4-a8cd-0050569b7e95,Rostock,13003,"Rostock, Hanse- und Universitätsstadt",130030000,"Rostock, Hanse- und Universitätsstadt",130030000000,Stadtmitte,...,Ausleihbibliothek,Umweltbibliothek Rostock,NABU Mittleres Mecklenburg e.V.,gewerblich-privat,NaN,Do 12:00-17:00 oder nach vorheriger Absprache,+49 381 4903162,NaN,umweltbibo@nabu-mittleres-mecklenburg.de,https://www.nabu-mittleres-mecklenburg.de/umwe...
1,54.088325,12.120073,7a99639c-4d54-11e4-a8cc-0050569b7e95,Rostock,13003,"Rostock, Hanse- und Universitätsstadt",130030000,"Rostock, Hanse- und Universitätsstadt",130030000000,Kröpeliner-Tor-Vorstadt,...,Ausleihbibliothek,Spezialsammlung des rat+tat e.V.,rat+tat Rostock e.V.,gewerblich-privat,NaN,NaN,+49 381 453156,NaN,post@ratundtat-rostock.de,https://ratundtat-rostock.de/
2,54.082161,12.137616,8623ecfa-494e-11e4-b78b-0050569b7e95,Rostock,13003,"Rostock, Hanse- und Universitätsstadt",130030000,"Rostock, Hanse- und Universitätsstadt",130030000000,Stadtmitte,...,Präsenzbibliothek,Universitätsbibliothek Rostock – Wossidlo-Archiv,Land Mecklenburg-Vorpommern,öffentlich,0.0,nur nach vorheriger Absprache,+49 381 4988701,NaN,stefan.siebert@uni-rostock.de,https://www.ub.uni-rostock.de/standorte-einric...
3,54.087312,12.133249,8eac7edc-4971-11e4-9bfd-0050569b7e95,Rostock,13003,"Rostock, Hanse- und Universitätsstadt",130030000,"Rostock, Hanse- und Universitätsstadt",130030000000,Stadtmitte,...,Präsenzbibliothek,Kempowski Archiv Rostock,Ein bürgerliches Haus e.V.,gewerblich-privat,NaN,Di-So 14:00-17:00,+49 381 2037540,NaN,kempowski-archiv-rostock@t-online.de,https://kempowski-archiv-rostock.de/
4,54.081308,12.127929,9cfb1aca-496c-11e4-9bfb-0050569b7e95,Rostock,13003,"Rostock, Hanse- und Universitätsstadt",130030000,"Rostock, Hanse- und Universitätsstadt",130030000000,Stadtmitte,...,Präsenzbibliothek,Mediathek des Instituts franco-allemand de Ros...,Institut franco-allemand de Rostock e.V.,gewerblich-privat,NaN,NaN,+49 381 455007,NaN,info@ifa-rostock.de,https://www.institutfrancais.de/rostock/mediat...


Dropping unnecessary columns

In [130]:
df = df.drop(columns=['uuid', 'kreis_schluessel', 'kreis_name', 'gemeindeverband_name', 'gemeindeverband_schluessel', 'gemeinde_name', 'gemeinde_schluessel', 'gemeindeteil_name', 'art', 'traeger_bezeichnung', 'traeger_art', 'barrierefrei', 'oeffnungszeiten', 'telefon_festnetz', 'telefon_mobil', 'email', 'gemeindeteil_schluessel', 'strasse_name', 'strasse_schluessel', 'hausnummer', 'hausnummer_zusatz', 'postleitzahl', 'website'])
df.head()

,latitude,longitude,bezeichnung
0,54.085840,12.135889,Umweltbibliothek Rostock
1,54.088325,12.120073,Spezialsammlung des rat+tat e.V.
2,54.082161,12.137616,Universitätsbibliothek Rostock – Wossidlo-Archiv
3,54.087312,12.133249,Kempowski Archiv Rostock
4,54.081308,12.127929,Mediathek des Instituts franco-allemand de Ros...


Create a NamedPoint class that contains the necessary data for the libraries

In [131]:
class NamedPoint():
    def __init__(self, name, x, y):
        self.X = x
        self.Y = y
        self.name = name

Organizing the data of the libraries into a list

In [132]:
locations = []

for location in df.iterrows():
    point = NamedPoint(location[1]['bezeichnung'], location[1]['longitude'], location[1]['latitude'])
    locations.append(point)

In [133]:
print(f"There are {len(locations)} libraries in Rostock")

There are 24 libraries in Rostock


Displaying the data for all libraries

In [134]:
data_dict = {'Name': [point.name for point in locations],
             'X': [point.X for point in locations],
             'Y': [point.Y for point in locations]}

dataframe = pd.DataFrame(data_dict)
dataframe

,Name,X,Y
0,Umweltbibliothek Rostock,12.135889,54.085840
1,Spezialsammlung des rat+tat e.V.,12.120073,54.088325
2,Universitätsbibliothek Rostock – Wossidlo-Archiv,12.137616,54.082161
3,Kempowski Archiv Rostock,12.133249,54.087312
4,Mediathek des Instituts franco-allemand de Ros...,12.127929,54.081308
5,Bibliothek des Bundesamtes für Seeschifffahrt ...,12.106173,54.097205
6,Stadtbibliothek Rostock – Zentralbibliothek,12.136717,54.088553
7,Stadtbibliothek Rostock – Kurbibliothek Warnem...,12.079314,54.178185
8,Stadtbibliothek Rostock – Stadtteilbibliothek ...,12.079399,54.099026
9,Universitätsbibliothek Rostock – Patent- und N...,12.112792,54.084226


Displaying libraries on the map

In [135]:
map_osm = folium.Map(location=[54.100988, 12.092044], zoom_start=11)
for location in locations:
    longitude = location.X
    latitude = location.Y
    folium.Marker([latitude, longitude]).add_to(map_osm)
map_osm

Using the KMeans model to determine coffee shops

In [136]:
number_of_shops = 3

kmeans = KMeans(n_clusters=number_of_shops, random_state=42)
kmeans.fit(df[['latitude', 'longitude']])

coffee_shop_locations = pd.DataFrame(kmeans.cluster_centers_, columns=['Latitude', 'Longitude'])

c:\Users\LazlowG\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


The coordinates of the coffee shops

In [137]:
for _, row in coffee_shop_locations.iterrows():
    print(f"{row['Latitude']}, {row['Longitude']}")

54.08774758379922, 12.13831209187672
54.15948198256975, 12.073929586016124
54.08885621904594, 12.10439751954858


Displaying libraries and coffe shops on the map. (red: coffe shops, blue: libraries)

In [138]:
map = folium.Map(location=[54.100988, 12.092044], zoom_start=11)

for _, row in coffee_shop_locations.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    folium.Marker([latitude, longitude], icon=folium.Icon(color='red', icon='info-sign')).add_to(map)

for location in locations:
    longitude = location.X
    latitude = location.Y
    folium.Marker([latitude, longitude]).add_to(map)

map